In [2]:
import json
import numpy as np

path = "./tweet_data/"

training_files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]
path2 = "./test_data/"
testing_files = ["sample1_period1.txt", "sample2_period2.txt", "sample3_period3.txt", "sample4_period1.txt", "sample5_period1.txt", "sample6_period2.txt", "sample7_period3.txt", "sample8_period1.txt", "sample9_period2.txt", "sample10_period3.txt"]
for i, fl in enumerate(training_files):
    print "files[" + str(i) + "] => " + fl
    

    

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [3]:
# transform the 2015 Feb. 1, 8:00 am and 8:00 pm into timestamp form
import datetime, time
import pytz

start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))
# used as the zero point of the time
base_time = time.mktime(time.strptime("2015-01-01 00:00:00",'%Y-%m-%d %H:%M:%S'))

In [4]:
def floor_date(date):
    return date // 3600

In [5]:
# the extracted data contains
# 0. the citation date 1. the favourite_count 2. the followers number
# 3. the length of the title 4. the number of the twitters
def load_file(file):
    '''
    load the time, follows, and retweets
    '''
    # 0. before Feb. 1, 8:00 a.m. 1. between 2. after 8:00 p.m.
    result = [[], [], []]
    for line in open(path + file, 'r') :
        tmp = []
        a = json.loads(line)
        citation_date = a['citation_date']
        tmp.append(int((citation_date - base_time) // 3600))
        tmp.append(a['author']['followers'])
        tmp.append(a['tweet']['user']['favourites_count'])
        tmp.append(len(a['title']))
        tmp.append(1)
        if citation_date < start_time:
            result[0].append(tmp)
        elif citation_date < end_time:
            result[1].append(tmp)
        else:
            result[2].append(tmp)
    return result
def load_test_file(file):
    '''
    load the time, follows, and retweets
    '''
    # 0. before Feb. 1, 8:00 a.m. 1. between 2. after 8:00 p.m.
    result = [[], [], []]
    for line in open(path2 + file, 'r') :
        tmp = []
        a = json.loads(line)
        citation_date = a['citation_date']
        tmp.append(int((citation_date - base_time) // 3600))
        tmp.append(a['author']['followers'])
        tmp.append(a['tweet']['user']['favourites_count'])
        tmp.append(len(a['title']))
        tmp.append(1)
        if citation_date < start_time:
            result[0].append(tmp)
        elif citation_date < end_time:
            result[1].append(tmp)
        else:
            result[2].append(tmp)
    return result

In [101]:
def group_data(dataset):
    '''
    parameter dataset is one of the three time slot defined above, namely, data[0], data[1], data[2]
    count all the features in 1-hour slot
    
    return the train feature (train_set[0:4]) and the result (train_set[5])
    '''
    # get the maximum and minimum time
    max_time = dataset[0][0]
    min_time = dataset[0][0]
    for p in dataset:
        time = p[0]
        if max_time < time:
            max_time = time
        if min_time > time:
            min_time = time
    
    train_set = []
    
    for i in range(min_time, max_time + 1):
        train_set.append([0] * 5)
    
    for p in dataset:
        index = p[0] - min_time
        train_set[index][0] = p[0]
        for i in range(1, 5):
            train_set[index][i] += p[i]
    
    result = []
    for i in range(0, max_time - min_time):
        result.append(train_set[i + 1][4])
    train_all = []
    for dset in train_set:
        train_all.append(dset)
    return train_all[:-1], result


In [6]:
def get_error(predict, target):
    '''
        used to get the average error between predict and target
    '''
    length = len(target)
    return sum([abs(predict[i] - target[i]) for i in range(length)]) / float(length)

In [ ]:
# using k-fold to divide the train and test set
from sklearn.svm import LinearSVR
svm = LinearSVR(random_state = 0)
time_range = {0: "Before Feb. 1, 8:00 a.m. ", 1: "Between Feb. 1, 8:00 a.m. and 8:00 p.m. ", \
              2: "After Feb. 1, 8:00 p.m."}
final_result = []
train_model = svm
for f in training_files:
    file_data = load_file(f)
for f in testing_files:
    file_data2 = load_test_file(f)

for i in range(3):
    train_all, result  = group_data(file_data[i])
    test_all, result2  = group_data(file_data2[i])  
    model = train_model.fit(train_all, result)
    predict = model.predict(test_all)
    error += get_error(predict, result2)
    a = "the file is " + f + " , the time range is " + time_range[i] + \
    " and the error is \n" + str(get_error(train_all, result, svm))
    final_result.append(a)

In [65]:
for x in final_result:
    print x

In [75]:
train_all

[312,
 3181.0,
 25125,
 723,
 8,
 313,
 13031.0,
 11547,
 780,
 7,
 314,
 1406539.0,
 20855,
 1373,
 12,
 315,
 26140.0,
 1290,
 1364,
 12,
 316,
 29638.0,
 21276,
 1534,
 14,
 317,
 39735.0,
 9737,
 1347,
 11,
 318,
 39514.0,
 33173,
 2258,
 21,
 319,
 226562.0,
 69967,
 3811,
 32,
 320,
 822461.0,
 62619,
 7094,
 61,
 321,
 177967.0,
 147291,
 6847,
 58,
 322,
 582835.0,
 31130,
 7912,
 70,
 323,
 266742.0,
 76820,
 6590,
 57,
 324,
 178891.0,
 127300,
 6188,
 56,
 325,
 250050.0,
 78540,
 7286,
 63,
 326,
 260018.0,
 81171,
 6735,
 56,
 327,
 77429.0,
 152036,
 3862,
 34,
 328,
 84394.0,
 44432,
 2836,
 25,
 329,
 556673.0,
 24610,
 2886,
 24,
 330,
 657680.0,
 11295,
 2135,
 19,
 331,
 49625.0,
 54987,
 3843,
 35,
 332,
 645764.0,
 45627,
 3482,
 32,
 333,
 40053.0,
 24571,
 2129,
 21,
 334,
 10036.0,
 6476,
 1992,
 16,
 335,
 10989.0,
 8586,
 1582,
 14,
 336,
 8339.0,
 16974,
 1385,
 11,
 337,
 229951.0,
 15289,
 1874,
 16,
 338,
 5174.0,
 5776,
 1017,
 9,
 339,
 35330.0,
 88140,
